# Phase 1: Core Pipeline - Educational Walkthrough

This notebook walks through the complete Phase 1 pipeline step-by-step.

**What you'll learn:**
1. How to fetch emails from Gmail API
2. How to parse HTML newsletters
3. How to extract events using LLMs
4. How to generate embeddings
5. How to store in Qdrant vector database
6. How to search semantically

**Prerequisites:**
- Phase 0 complete (environment setup)
- Gmail OAuth configured
- Qdrant running in Docker
- OpenAI API key set in `.env`

## Setup: Import Libraries and Initialize

First, let's import all the modules we'll need and set up our environment.

In [ ]:
# Standard library imports
import sys
import json
import os
import uuid
from datetime import datetime
from pathlib import Path

# IMPORTANT: Change working directory to project root
# This ensures credentials.json and token.json are found correctly
project_root = Path('/Users/Shared/ALL WORKSPACE/Hackathons/mom_hack/feedprism_main')
os.chdir(project_root)
print(f"📁 Working directory: {os.getcwd()}")

# Add project root to Python path
sys.path.insert(0, str(project_root))

# Third-party imports
import asyncio
from loguru import logger

# Our custom modules (we'll build these step by step)
from app.config import settings
from app.services.gmail_client import GmailClient
from app.services.parser import EmailParser
from app.services.extractor import LLMExtractor
from app.services.embedder import EmbeddingService
from app.database.qdrant_client import QdrantService
from app.models.extraction import ExtractedEvent, EventExtractionResult

# Configure logger for notebook
logger.remove()  # Remove default handler
logger.add(sys.stdout, level="INFO", format="<level>{level: <8}</level> | {message}")

print("✅ All imports successful!")
print(f"📁 Project root: {project_root}")
print(f"🔑 Credentials path: {settings.gmail_credentials_path}")
print(f"🎫 Token path: {settings.gmail_token_path}")
print(f"🔧 Config loaded: OpenAI key = {'***' + settings.openai_api_key[-4:]}")

## Step 1: Fetch Emails from Gmail

**What's happening:**
- We connect to Gmail API using OAuth credentials
- Search for content-rich emails (newsletters, events)
- Fetch full email content including HTML body

**Key concepts:**
- Gmail API uses labels and search queries
- Emails have multipart structure (text + HTML)
- We prefer HTML for newsletters (richer content)

In [ ]:
# Initialize Gmail client
# This will:
# 1. Load OAuth credentials from token.json
# 2. Refresh token if expired
# 3. Connect to Gmail API
gmail_client = GmailClient()

print("✅ Gmail client initialized")
print("\n📧 Fetching emails...")

# Fetch content-rich emails from last 7 days
# Parameters:
#   days_back: How far back to search
#   max_results: Maximum emails to fetch
emails = gmail_client.fetch_content_rich_emails(
    days_back=7,
    max_results=5  # Start with just 5 for this demo
)

print(f"\n✅ Fetched {len(emails)} emails")

# Let's examine the first email
if emails:
    first_email = emails[0]
    print("\n📬 First Email Details:")
    print(f"  Subject: {first_email['subject']}")
    print(f"  From: {first_email['from']}")
    print(f"  Date: {first_email['date']}")
    print(f"  Has HTML: {first_email['body_html'] is not None}")
    print(f"  HTML Length: {len(first_email['body_html']) if first_email['body_html'] else 0} characters")
    print(f"  Snippet: {first_email['snippet'][:100]}...")
else:
    print("⚠️ No emails found. Try adjusting search criteria.")

## Step 2: Parse HTML Email

**What's happening:**
- Take raw HTML from email
- Remove tracking pixels, scripts, styles
- Convert to clean text while preserving structure
- Extract all links

**Why this matters:**
- Newsletters have complex HTML with inline CSS
- Tracking pixels and ads add noise
- Clean text is better for LLM extraction
- Links contain important registration/info URLs

In [ ]:
# Initialize HTML parser
# This uses BeautifulSoup4 + html2text
parser = EmailParser()

print("✅ Email parser initialized")

# Parse the first email's HTML
if emails and emails[0]['body_html']:
    print("\n🔍 Parsing HTML...")
    
    parsed_result = parser.parse_html_email(emails[0]['body_html'])
    
    print(f"\n✅ Parsing complete!")
    print(f"\n📄 Results:")
    print(f"  Title: {parsed_result['title']}")
    print(f"  Clean text length: {len(parsed_result['text'])} characters")
    print(f"  Links extracted: {len(parsed_result['links'])}")
    
    # Show first few links
    print(f"\n🔗 Sample Links:")
    for i, link in enumerate(parsed_result['links'][:3], 1):
        print(f"  {i}. {link['text'][:50]}... → {link['url'][:60]}...")
    
    # Show preview of clean text
    print(f"\n📝 Clean Text Preview (first 500 chars):")
    print("-" * 70)
    print(parsed_result['text'][:500])
    print("-" * 70)
    
    # Store for next step
    clean_text = parsed_result['text']
    email_subject = emails[0]['subject']
else:
    print("⚠️ No HTML body to parse")

## Step 3: Extract Events with LLM

**What's happening:**
- Send clean text to OpenAI GPT-4o-mini
- Use structured output (JSON Schema) for guaranteed valid JSON
- Extract event details: title, date, location, link, etc.
- Get confidence score

**How structured output works:**
1. We define a Pydantic model (ExtractedEvent)
2. Pydantic generates JSON Schema
3. OpenAI guarantees response matches schema
4. No parsing errors or invalid JSON!

**Note:** This step requires OpenAI API key and will make an API call (~$0.0003 per email)

In [ ]:
# Initialize LLM extractor
# This creates an async OpenAI client
extractor = LLMExtractor()

print("✅ LLM extractor initialized")
print(f"   Model: {extractor.model}")
print(f"   Temperature: {extractor.temperature} (0 = deterministic)")

# Extract events from the parsed text
# This is an async function, so we need to use await
print("\n🤖 Extracting events with LLM...")
print("   (This may take 2-4 seconds)")

# Run async extraction
extraction_result = await extractor.extract_events(
    email_text=clean_text,
    email_subject=email_subject
)

print(f"\n✅ Extraction complete!")
print(f"\n📊 Results:")
print(f"  Events found: {len(extraction_result.events)}")
print(f"  Confidence: {extraction_result.confidence:.2f} (0.0 - 1.0)")

# Display each extracted event
if extraction_result.events:
    print(f"\n🎉 Extracted Events:")
    print("=" * 70)
    
    for i, event in enumerate(extraction_result.events, 1):
        print(f"\nEvent {i}:")
        print(f"  📌 Title: {event.title}")
        print(f"  📝 Description: {event.description[:100] if event.description else 'N/A'}...")
        print(f"  📅 Start Time: {event.start_time or 'Not specified'}")
        print(f"  🕒 Timezone: {event.timezone or 'Not specified'}")
        print(f"  📍 Location: {event.location or 'Not specified'}")
        print(f"  🔗 Registration: {event.registration_link or 'Not specified'}")
        print(f"  💰 Cost: {event.cost or 'Not specified'}")
        print(f"  🏢 Organizer: {event.organizer or 'Not specified'}")
        print(f"  🏷️  Tags: {', '.join(event.tags) if event.tags else 'None'}")
        print(f"  ⏰ Status: {event.compute_status().value}")
    
    print("=" * 70)
else:
    print("\n⚠️ No events found in this email")

## Step 4: Generate Embeddings

**What's happening:**
- Convert event text to dense vector (384 dimensions)
- Use sentence-transformers (all-MiniLM-L6-v2)
- Vectors capture semantic meaning

**Why embeddings:**
- Enable semantic search ("machine learning workshop" finds "ML training")
- Similar events have similar vectors
- Much better than keyword matching

**Model details:**
- Runs locally (no API costs)
- ~50ms per embedding
- First run downloads model (~80MB)

In [ ]:
# Initialize embedding service
# First run will download the model (takes ~30 seconds)
# Subsequent runs load from cache (~2 seconds)
print("🔄 Loading embedding model...")
print("   (First time: downloads ~80MB model)")
print("   (Subsequent: loads from cache)")

embedder = EmbeddingService()

print(f"\n✅ Embedding service initialized")
print(f"   Model: {embedder.model_name}")
print(f"   Dimension: {embedder.dimension}D vectors")

# Generate embeddings for each event
if extraction_result.events:
    print(f"\n🔢 Generating embeddings for {len(extraction_result.events)} events...")
    
    event_embeddings = []
    
    for i, event in enumerate(extraction_result.events, 1):
        # Combine title and description for richer embedding
        text_to_embed = f"{event.title} {event.description or ''}"
        
        # Generate embedding
        vector = embedder.embed_text(text_to_embed)
        
        event_embeddings.append({
            'event': event,
            'vector': vector,
            'text': text_to_embed
        })
        
        print(f"  Event {i}: Generated {len(vector)}D vector")
    
    # Show example vector (first 10 dimensions)
    print(f"\n📊 Example Vector (first 10 dimensions):")
    print(f"   {event_embeddings[0]['vector'][:10]}")
    print(f"   ... (374 more dimensions)")
    
    # Demonstrate semantic similarity
    if len(event_embeddings) >= 2:
        import numpy as np
        
        vec1 = np.array(event_embeddings[0]['vector'])
        vec2 = np.array(event_embeddings[1]['vector'])
        
        # Cosine similarity
        similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
        
        print(f"\n🔍 Semantic Similarity Example:")
        print(f"   Event 1: {event_embeddings[0]['event'].title}")
        print(f"   Event 2: {event_embeddings[1]['event'].title}")
        print(f"   Similarity: {similarity:.3f} (1.0 = identical, 0.0 = unrelated)")
else:
    print("\n⚠️ No events to embed")

## Step 5: Store in Qdrant Vector Database

**What's happening:**
- Connect to Qdrant (running in Docker)
- Create collection if doesn't exist
- Store vectors + metadata (payload)
- Build HNSW index for fast search

**Qdrant concepts:**
- **Collection**: Container for vectors (like a table)
- **Point**: Single vector + payload
- **Payload**: Metadata (title, date, location, etc.)
- **HNSW**: Fast approximate nearest neighbor search

**Why Qdrant:**
- Built for vector search
- Payload filtering ("only upcoming events")
- Fast (millisecond queries)
- Free and runs locally

In [ ]:
# Initialize Qdrant client
# This connects to Docker container at localhost:6333
qdrant = QdrantService()

print("✅ Qdrant client initialized")
print(f"   Host: {qdrant.host}:{qdrant.port}")
print(f"   Collection: {qdrant.collection_name}")

# Create collection (if doesn't exist)
print("\n🗄️  Creating/verifying collection...")
qdrant.create_collection(recreate=False)  # Don't delete existing data

# Get collection info
info = qdrant.get_collection_info()
print(f"\n📊 Collection Info:")
print(f"   Name: {info['name']}")
print(f"   Points: {info['points_count']}")
print(f"   Status: {info['status']}")

# Prepare points for insertion
if event_embeddings:
    from qdrant_client.models import PointStruct
    
    print(f"\n💾 Preparing {len(event_embeddings)} points for storage...")
    
    points = []
    
    for item in event_embeddings:
        event = item['event']
        vector = item['vector']
        
        # Create point with unique ID
        point = PointStruct(
            id=str(uuid.uuid4()),  # Unique identifier
            vector=vector,  # 384D embedding
            payload={  # Metadata for filtering and display
                'title': event.title,
                'description': event.description,
                'start_time': event.start_time,
                'end_time': event.end_time,
                'timezone': event.timezone,
                'location': event.location,
                'registration_link': str(event.registration_link) if event.registration_link else None,
                'tags': event.tags,
                'organizer': event.organizer,
                'cost': event.cost,
                'status': event.compute_status().value,
                'source_email_id': emails[0]['id'],
                'source_subject': emails[0]['subject'],
                'source_from': emails[0]['from'],
                'type': 'event',
                'extracted_at': datetime.now().isoformat()
            }
        )
        
        points.append(point)
        print(f"  ✓ Prepared: {event.title}")
    
    # Insert into Qdrant
    print(f"\n📤 Upserting points to Qdrant...")
    qdrant.upsert_points(points)
    
    print(f"\n✅ Successfully stored {len(points)} events!")
    
    # Get updated collection info
    updated_info = qdrant.get_collection_info()
    print(f"\n📊 Updated Collection:")
    print(f"   Total points: {updated_info['points_count']}")
else:
    print("\n⚠️ No events to store")

## Step 6: Semantic Search

**What's happening:**
- Convert search query to vector
- Find similar vectors in Qdrant
- Return events ranked by similarity
- Apply filters (type, status, etc.)

**How it works:**
1. Query: "machine learning workshop" → vector
2. Qdrant finds nearest neighbor vectors
3. Returns events with similarity scores
4. Higher score = more relevant

**Magic of semantic search:**
- "ML training" matches "machine learning workshop"
- "AI conference" matches "artificial intelligence summit"
- Works across synonyms and related concepts

In [ ]:
# Let's try some searches!
print("🔍 Semantic Search Demo\n")
print("=" * 70)

# Define search queries
search_queries = [
    "machine learning workshop",
    "AI conference",
    "data science event",
    "online webinar"
]

for query in search_queries:
    print(f"\n🔎 Query: '{query}'")
    print("-" * 70)
    
    # Generate query vector
    query_vector = embedder.embed_text(query)
    
    # Search Qdrant
    # Parameters:
    #   query_vector: The embedding of our search query
    #   limit: How many results to return
    #   filter_dict: Optional filters (e.g., only upcoming events)
    results = qdrant.search(
        query_vector=query_vector,
        limit=3,
        filter_dict={'type': 'event'}  # Only return events
    )
    
    if results:
        print(f"Found {len(results)} results:\n")
        
        for i, result in enumerate(results, 1):
            payload = result['payload']
            score = result['score']
            
            print(f"{i}. {payload['title']}")
            print(f"   📊 Relevance: {score:.3f} (0.0 - 1.0)")
            print(f"   📍 Location: {payload.get('location', 'N/A')}")
            print(f"   📅 Start: {payload.get('start_time', 'N/A')}")
            print(f"   ⏰ Status: {payload.get('status', 'N/A')}")
            print(f"   🔗 Link: {payload.get('registration_link', 'N/A')[:60]}...")
            print()
    else:
        print("No results found\n")

print("=" * 70)

## Step 7: Advanced Search with Filters

**Payload filtering:**
- Search within specific criteria
- Combine semantic search + filters
- Much more powerful than search alone

**Examples:**
- "ML events" + only upcoming
- "conferences" + only free
- "workshops" + only online

In [ ]:
print("🎯 Filtered Search Demo\n")
print("=" * 70)

# Search 1: Only upcoming events
print("\n🔎 Search: 'AI event' (upcoming only)")
print("-" * 70)

query_vec = embedder.embed_text("AI event")
results = qdrant.search(
    query_vector=query_vec,
    limit=5,
    filter_dict={
        'type': 'event',
        'status': 'upcoming'  # Only future events
    }
)

print(f"Found {len(results)} upcoming events\n")
for i, r in enumerate(results, 1):
    print(f"{i}. {r['payload']['title']} (score: {r['score']:.3f})")

# Search 2: Only online events
print("\n🔎 Search: 'workshop' (online only)")
print("-" * 70)

query_vec = embedder.embed_text("workshop")
results = qdrant.search(
    query_vector=query_vec,
    limit=5,
    filter_dict={
        'type': 'event',
        'location': 'Online'  # Only virtual events
    }
)

print(f"Found {len(results)} online workshops\n")
for i, r in enumerate(results, 1):
    print(f"{i}. {r['payload']['title']} (score: {r['score']:.3f})")

print("\n" + "=" * 70)

## Summary: Complete Pipeline

**What we accomplished:**

1. ✅ **Fetched emails** from Gmail API
2. ✅ **Parsed HTML** to clean text
3. ✅ **Extracted events** using LLM structured output
4. ✅ **Generated embeddings** for semantic search
5. ✅ **Stored in Qdrant** with rich metadata
6. ✅ **Searched semantically** with filters

**Key learnings:**

- **Gmail API**: OAuth authentication, multipart emails
- **HTML Parsing**: BeautifulSoup4, tracking pixel removal
- **LLM Extraction**: Structured output guarantees valid JSON
- **Embeddings**: 384D vectors capture semantic meaning
- **Qdrant**: Vector database with payload filtering
- **Semantic Search**: Find by meaning, not just keywords

**Performance:**

- Email fetch: ~3s for 5 emails
- HTML parsing: ~50ms per email
- Event extraction: ~2-3s per email
- Embedding: ~50ms per event
- Qdrant storage: ~200ms for batch
- Search: ~100ms per query

**Next steps:**

- **Phase 2**: Add courses and blogs extraction
- **Phase 3**: Implement hybrid search (dense + sparse)
- **Phase 4**: Advanced Qdrant features (grouping, discovery)
- **Phase 5**: Deduplication and temporal reasoning

## Bonus: Process Multiple Emails

Now that you understand each step, let's process all fetched emails in a loop!

In [ ]:
print("🔄 Processing All Emails\n")
print("=" * 70)

total_events = 0
all_points = []

for i, email in enumerate(emails, 1):
    print(f"\n[{i}/{len(emails)}] Processing: {email['subject'][:60]}...")
    
    try:
        # Skip if no HTML
        if not email['body_html']:
            print("  ⚠️ No HTML body, skipping")
            continue
        
        # Parse
        parsed = parser.parse_html_email(email['body_html'])
        print(f"  ✓ Parsed: {len(parsed['text'])} chars, {len(parsed['links'])} links")
        
        # Extract
        result = await extractor.extract_events(parsed['text'], email['subject'])
        print(f"  ✓ Extracted: {len(result.events)} events (confidence: {result.confidence:.2f})")
        
        if not result.events:
            continue
        
        # Embed and create points
        for event in result.events:
            text = f"{event.title} {event.description or ''}"
            vector = embedder.embed_text(text)
            
            from qdrant_client.models import PointStruct
            point = PointStruct(
                id=str(uuid.uuid4()),
                vector=vector,
                payload={
                    'title': event.title,
                    'description': event.description,
                    'start_time': event.start_time,
                    'location': event.location,
                    'registration_link': str(event.registration_link) if event.registration_link else None,
                    'tags': event.tags,
                    'status': event.compute_status().value,
                    'source_email_id': email['id'],
                    'source_subject': email['subject'],
                    'type': 'event'
                }
            )
            all_points.append(point)
        
        total_events += len(result.events)
        print(f"  ✓ Created {len(result.events)} points")
        
    except Exception as e:
        print(f"  ❌ Error: {e}")
        continue

# Store all points
if all_points:
    print(f"\n💾 Storing {len(all_points)} total events...")
    qdrant.upsert_points(all_points)
    print(f"✅ Complete!")

print(f"\n" + "=" * 70)
print(f"📊 Final Statistics:")
print(f"   Emails processed: {len(emails)}")
print(f"   Events extracted: {total_events}")
print(f"   Points stored: {len(all_points)}")

# Final collection info
final_info = qdrant.get_collection_info()
print(f"   Total in database: {final_info['points_count']} points")
print("=" * 70)